In [ ]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import scipy
import sklearn
import itertools 
from itertools import cycle 
import os.path as op
import timeit 
import json
import math


In [ ]:
import multiprocessing as m_proc
m_proc.cpu_count()

In [ ]:
# Import MDAnalysis
import MDAnalysis as mda
import statsmodels as stats

## PLGA-PEG/acetone only systems

In [ ]:
n_plgapeg = np.array([6,8,10,16,20,30])

In [ ]:
# Load dataframe from pickled pandas object
rgace_nplgapeg = pd.read_pickle("plga_peg_ace/PLGAPEG_ace_Rg.pkl")
rgace_nplgapeg = rgace_nplgapeg.rename(columns={r"$R_{g}$ [Angstrom] N = 30 PEG DMSO ": r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG acetone"})
rgace_nplgapeg

In [ ]:
lpace_plgapeg = pd.read_pickle("plga_peg_ace/PLGAPEG_ace_Lp.pkl")
lpace_plgapeg = lpace_plgapeg.rename(columns={r"$L_{p}$ [Angstrom] N = 30 PEG DMSO ": r"$L_{p}$ [Angstrom] N = 30 PLGA-PEG acetone"})
lpace_plgapeg

In [ ]:
lpace_plgapeg[r"$L_{p}$ [Angstrom] N = 6 PLGA-PEG acetone"]

In [ ]:
lpace_avgplgapeg = np.array([np.mean(lpace_plgapeg[r"$L_{p}$ [Angstrom] N = 6 PLGA-PEG acetone"])
                      ,np.mean(lpace_plgapeg[r"$L_{p}$ [Angstrom] N = 8 PLGA-PEG acetone"])
                     ,np.mean(lpace_plgapeg[r"$L_{p}$ [Angstrom] N = 10 PLGA-PEG acetone"])
                    ,np.mean(lpace_plgapeg[r"$L_{p}$ [Angstrom] N = 16 PLGA-PEG acetone"])
                     ,np.mean(lpace_plgapeg[r"$L_{p}$ [Angstrom] N = 20 PLGA-PEG acetone"])
                     ,np.mean(lpace_plgapeg[r"$L_{p}$ [Angstrom] N = 30 PLGA-PEG acetone"])])
lpace_avgplgapeg

In [ ]:
np.mean(lpace_avgplgapeg)

In [ ]:
np.std(lpace_avgplgapeg)

In [ ]:
rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG acetone"]

In [ ]:
rgace_npavgplgapeg = np.array([np.mean(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 6 PLGA-PEG acetone"])
                      ,np.mean(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 8 PLGA-PEG acetone"])
                     ,np.mean(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 10 PLGA-PEG acetone"])
                    ,np.mean(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 16 PLGA-PEG acetone"])
                     ,np.mean(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 20 PLGA-PEG acetone"])
                     ,np.mean(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG acetone"])])
rgace_npavgplgapeg

In [ ]:
rgace_stdplgapeg = np.array([np.std(np.log10(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 6 PLGA-PEG acetone"]))
                      ,np.std(np.log10(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 8 PLGA-PEG acetone"]))
                     ,np.std(np.log10(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 10 PLGA-PEG acetone"]))
                    ,np.std(np.log10(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 16 PLGA-PEG acetone"]))
                     ,np.std(np.log10(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 20 PLGA-PEG acetone"]))
                     ,np.std(np.log10(rgace_nplgapeg[r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG acetone"]))])
rgace_stdplgapeg

In [ ]:
np.log10(n_plgapeg)

In [ ]:
# From fitting all points, I get best fit 
from sklearn.linear_model import LinearRegression
model_vppace = LinearRegression(fit_intercept=True)
model_vppace.fit(np.log10(n_plgapeg).reshape(-1,1), np.log10(rgace_npavgplgapeg))
# Slope here is in nanometers
print("Model slope:    ", model_vppace.coef_[0])
print("Model intercept:", model_vppace.intercept_)

In [ ]:
gg_pace = model_vppace.predict(np.log10(n_plgapeg.reshape(-1,1)))
gg_pace

In [ ]:
# Residuals between the true y data and model y data 
resid_vppace = np.log10(rgace_npavgplgapeg) - gg_pace
resid_vppace

In [ ]:
# How to calculate mean squared error, N - 1 is dof 
mse_vppace = np.sum(resid_vppace**2)/(len(resid_vppace) - 1)
mse_vppace

In [ ]:
print("R2 score:", sklearn.metrics.r2_score(np.log10(rgace_npavgplgapeg), gg_pace))

In [ ]:
# How to calculate Sum((Xi - avg(X))^2): X values are the bond length values 
nt_ttppace = np.log10(n_plgapeg)
nt_ttppace -= np.mean(nt_ttppace)
nhui_ppace = nt_ttppace**2
np.sum(nhui_ppace)

In [ ]:
# How to calculate Sxy
hg_vppace = np.log10(rgace_npavgplgapeg) - np.mean(np.log10(rgace_npavgplgapeg))
ata_vppace = np.sum(hg_vppace*(np.log10(n_plgapeg) - np.mean(np.log10(n_plgapeg))))
ata_vppace

In [ ]:
# Test statistic, t0
(ata_vppace/np.sum(nhui_ppace))/np.sqrt(mse_vppace/(np.sum(nhui_ppace)))

In [ ]:
# t-value with 95 % confidence intervals 
scipy.stats.t.ppf(0.95, len(resid_vppace)-1)

In [ ]:
# How to calculate 95% confidence interval for the slope 
flc_vppace = scipy.stats.t.ppf(0.95, len(resid_vppace)-1)*np.sqrt(mse_vppace/(np.sum(nhui_ppace)))
flc_vppace

## PLGA-PEG/DMSO only systems

In [ ]:
# Load dataframe from pickled pandas object
rgdmso_nplgapeg = pd.read_pickle("plga_peg_dmso/PLGAPEG_dmso_Rg.pkl")
rgdmso_nplgapeg = rgdmso_nplgapeg.rename(columns={r"$R_{g}$ [Angstrom] N = 30 PEG DMSO ": r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG DMSO"})
rgdmso_nplgapeg

In [ ]:
lpdmso_plgapeg = pd.read_pickle("plga_peg_dmso/PLGAPEG_dmso_Lp.pkl")
lpdmso_plgapeg = lpdmso_plgapeg.rename(columns={r"$L_{p}$ [Angstrom] N = 30 PEG DMSO ": r"$L_{p}$ [Angstrom] N = 30 PLGA-PEG DMSO"})
lpdmso_plgapeg

In [ ]:
lpdmso_plgapeg[r"$L_{p}$ [Angstrom] N = 6 PLGA-PEG DMSO"]

In [ ]:
lpdmso_avgplgapeg = np.array([np.mean(lpdmso_plgapeg[r"$L_{p}$ [Angstrom] N = 6 PLGA-PEG DMSO"])
                      ,np.mean(lpdmso_plgapeg[r"$L_{p}$ [Angstrom] N = 8 PLGA-PEG DMSO"])
                     ,np.mean(lpdmso_plgapeg[r"$L_{p}$ [Angstrom] N = 10 PLGA-PEG DMSO"])
                    ,np.mean(lpdmso_plgapeg[r"$L_{p}$ [Angstrom] N = 16 PLGA-PEG DMSO"])
                     ,np.mean(lpdmso_plgapeg[r"$L_{p}$ [Angstrom] N = 20 PLGA-PEG DMSO"])
                     ,np.mean(lpdmso_plgapeg[r"$L_{p}$ [Angstrom] N = 30 PLGA-PEG DMSO"])])
lpdmso_avgplgapeg

In [ ]:
np.mean(lpdmso_avgplgapeg)

In [ ]:
np.std(lpdmso_avgplgapeg)

In [ ]:
rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG DMSO"]

In [ ]:
rgdmso_npavgplgapeg = np.array([np.mean(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 6 PLGA-PEG DMSO"])
                      ,np.mean(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 8 PLGA-PEG DMSO"])
                     ,np.mean(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 10 PLGA-PEG DMSO"])
                    ,np.mean(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 16 PLGA-PEG DMSO"])
                     ,np.mean(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 20 PLGA-PEG DMSO"])
                     ,np.mean(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG DMSO"])])
rgdmso_npavgplgapeg

In [ ]:
rgdmso_stdplgapeg = np.array([np.std(np.log10(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 6 PLGA-PEG DMSO"]))
                      ,np.std(np.log10(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 8 PLGA-PEG DMSO"]))
                     ,np.std(np.log10(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 10 PLGA-PEG DMSO"]))
                    ,np.std(np.log10(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 16 PLGA-PEG DMSO"]))
                     ,np.std(np.log10(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 20 PLGA-PEG DMSO"]))
                     ,np.std(np.log10(rgdmso_nplgapeg[r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG DMSO"]))])
rgdmso_stdplgapeg

In [ ]:
np.log10(n_plgapeg)

In [ ]:
# From fitting all points, I get best fit 
from sklearn.linear_model import LinearRegression
model_vppdmso = LinearRegression(fit_intercept=True)
model_vppdmso.fit(np.log10(n_plgapeg).reshape(-1,1), np.log10(rgdmso_npavgplgapeg))
# Slope here is in nanometers
print("Model slope:    ", model_vppdmso.coef_[0])
print("Model intercept:", model_vppdmso.intercept_)

In [ ]:
gg_pdmso = model_vppdmso.predict(np.log10(n_plgapeg.reshape(-1,1)))
gg_pdmso

In [ ]:
# Residuals between the true y data and model y data 
resid_vppdmso = np.log10(rgdmso_npavgplgapeg) - gg_pdmso
resid_vppdmso

In [ ]:
# How to calculate mean squared error, N - 1 is dof 
mse_vppdmso = np.sum(resid_vppdmso**2)/(len(resid_vppdmso) - 1)
mse_vppdmso

In [ ]:
print("R2 score:", sklearn.metrics.r2_score(np.log10(rgdmso_npavgplgapeg), gg_pdmso))

In [ ]:
# How to calculate Sum((Xi - avg(X))^2): X values are the bond length values 
nt_ttppdmso = np.log10(n_plgapeg)
nt_ttppdmso -= np.mean(nt_ttppdmso)
nhui_ppdmso = nt_ttppdmso**2
np.sum(nhui_ppdmso)

In [ ]:
# How to calculate Sxy
hg_vppdmso = np.log10(rgdmso_npavgplgapeg) - np.mean(np.log10(rgdmso_npavgplgapeg))
ata_vppdmso = np.sum(hg_vppdmso*(np.log10(n_plgapeg) - np.mean(np.log10(n_plgapeg))))
ata_vppdmso

In [ ]:
# Test statistic, t0
(ata_vppdmso/np.sum(nhui_ppdmso))/np.sqrt(mse_vppdmso/(np.sum(nhui_ppdmso)))

In [ ]:
# t-value with 95 % confidence intervals 
scipy.stats.t.ppf(0.95, len(resid_vppdmso)-1)

In [ ]:
# How to calculate 95% confidence interval for the slope 
flc_vppdmso = scipy.stats.t.ppf(0.95, len(resid_vppdmso)-1)*np.sqrt(mse_vppdmso/(np.sum(nhui_ppdmso)))
flc_vppdmso

## PLGA-PEG/water only systems

In [ ]:
# Load dataframe from pickled pandas object
rgwat_nplgapeg = pd.read_pickle("plga_peg_wat/PLGAPEG_wat_Rg.pkl")
rgwat_nplgapeg = rgwat_nplgapeg.rename(columns={r"$R_{g}$ [Angstrom] N = 30 PEG DMSO ": r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG water"})
rgwat_nplgapeg

In [ ]:
lpwat_plgapeg = pd.read_pickle("plga_peg_wat/PLGAPEG_wat_Lp.pkl")
lpwat_plgapeg = lpwat_plgapeg.rename(columns={r"$L_{p}$ [Angstrom] N = 30 PEG DMSO ": r"$L_{p}$ [Angstrom] N = 30 PLGA-PEG water"})
lpwat_plgapeg

In [ ]:
lpwat_plgapeg[r"$L_{p}$ [Angstrom] N = 6 PLGA-PEG water"]

In [ ]:
lpwat_avgplgapeg = np.array([np.mean(lpwat_plgapeg[r"$L_{p}$ [Angstrom] N = 6 PLGA-PEG water"])
                      ,np.mean(lpwat_plgapeg[r"$L_{p}$ [Angstrom] N = 8 PLGA-PEG water"])
                     ,np.mean(lpwat_plgapeg[r"$L_{p}$ [Angstrom] N = 10 PLGA-PEG water"])
                    ,np.mean(lpwat_plgapeg[r"$L_{p}$ [Angstrom] N = 16 PLGA-PEG water"])
                     ,np.mean(lpwat_plgapeg[r"$L_{p}$ [Angstrom] N = 20 PLGA-PEG water"])
                     ,np.mean(lpwat_plgapeg[r"$L_{p}$ [Angstrom] N = 30 PLGA-PEG water"])])
lpwat_avgplgapeg

In [ ]:
np.mean(lpwat_avgplgapeg)

In [ ]:
np.std(lpwat_avgplgapeg)

In [ ]:
rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG water"]

### Fluory Exponents

In [ ]:
rgwat_npavgplgapeg = np.array([np.mean(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 6 PLGA-PEG water"])
                      ,np.mean(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 8 PLGA-PEG water"])
                     ,np.mean(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 10 PLGA-PEG water"])
                    ,np.mean(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 16 PLGA-PEG water"])
                     ,np.mean(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 20 PLGA-PEG water"])
                     ,np.mean(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG water"])])
rgwat_npavgplgapeg

In [ ]:
rgwat_stdplgapeg = np.array([np.std(np.log10(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 6 PLGA-PEG water"]))
                      ,np.std(np.log10(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 8 PLGA-PEG water"]))
                     ,np.std(np.log10(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 10 PLGA-PEG water"]))
                    ,np.std(np.log10(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 16 PLGA-PEG water"]))
                     ,np.std(np.log10(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 20 PLGA-PEG water"]))
                     ,np.std(np.log10(rgwat_nplgapeg[r"$R_{g}$ [Angstrom] N = 30 PLGA-PEG water"]))])
rgwat_stdplgapeg

In [ ]:
np.log10(n_plgapeg)

In [ ]:
# From fitting all points, I get best fit 
from sklearn.linear_model import LinearRegression
model_vppwat = LinearRegression(fit_intercept=True)
model_vppwat.fit(np.log10(n_plgapeg).reshape(-1,1), np.log10(rgwat_npavgplgapeg))
# Slope here is in nanometers
print("Model slope:    ", model_vppwat.coef_[0])
print("Model intercept:", model_vppwat.intercept_)

In [ ]:
gg_pwat = model_vppwat.predict(np.log10(n_plgapeg.reshape(-1,1)))
gg_pwat

In [ ]:
# Residuals between the true y data and model y data 
resid_vppwat = np.log10(rgwat_npavgplgapeg) - gg_pwat
resid_vppwat

In [ ]:
# How to calculate mean squared error, N - 1 is dof 
mse_vppwat = np.sum(resid_vppwat**2)/(len(resid_vppwat) - 1)
mse_vppwat

In [ ]:
print("R2 score:", sklearn.metrics.r2_score(np.log10(rgwat_npavgplgapeg), gg_pwat))

In [ ]:
# How to calculate Sum((Xi - avg(X))^2): X values are the bond length values 
nt_ttppwat = np.log10(n_plgapeg)
nt_ttppwat -= np.mean(nt_ttppwat)
nhui_ppwat = nt_ttppwat**2
np.sum(nhui_ppwat)

In [ ]:
# How to calculate Sxy
hg_vppwat = np.log10(rgwat_npavgplgapeg) - np.mean(np.log10(rgwat_npavgplgapeg))
ata_vppwat = np.sum(hg_vppwat*(np.log10(n_plgapeg) - np.mean(np.log10(n_plgapeg))))
ata_vppwat

In [ ]:
# Test statistic, t0
(ata_vppwat/np.sum(nhui_ppwat))/np.sqrt(mse_vppwat/(np.sum(nhui_ppwat)))

In [ ]:
# t-value with 95 % confidence intervals 
scipy.stats.t.ppf(0.95, len(resid_vppwat)-1)

In [ ]:
# How to calculate 95% confidence interval for the slope 
flc_vppwat= scipy.stats.t.ppf(0.95, len(resid_vppwat)-1)*np.sqrt(mse_vppwat/(np.sum(nhui_ppwat)))
flc_vppwat

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))

#plt.figure(figsize=(7,7))
ax.errorbar(np.log10(n_plgapeg), np.log10(rgace_npavgplgapeg), yerr=rgace_stdplgapeg, color='#0371EC', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
ax.errorbar(np.log10(n_plgapeg), np.log10(rgdmso_npavgplgapeg), yerr=rgdmso_stdplgapeg, color='#418D2B', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
#plt.errorbar(np.log10(n_plgapeg), np.log10(rgawmix_npavgplgapeg), yerr=rgawmix_stdplgapeg, color='#562A8B', linestyle="None",marker='o',
#     capsize=5, capthick=1, ecolor='black')
ax.errorbar(np.log10(n_plgapeg), np.log10(rgwat_npavgplgapeg), yerr=rgwat_stdplgapeg, color='#6E563A', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

ax.plot(np.log10(n_plgapeg), gg_pace, color='#0371EC')
ax.plot(np.log10(n_plgapeg), gg_pdmso, color='#418D2B')
#plt.plot(np.log10(n_plgapeg), gg_pawmix, color='#562A8B')
ax.plot(np.log10(n_plgapeg), gg_pwat, color='#6E563A')

arfont = {'fontname':'Arial'}
#hfont = {'fontname':'Helvetica'}


ax.set_title(r'Fluory Exponent', fontsize=16, **arfont)
ax.set_xlabel(r'Log($N_{PLGA-PEG}$)', fontsize=16, **arfont)
ax.set_ylabel(r'Log($R_{g}$)', fontsize=16, **arfont)
ax.set_xlim(0.7,1.6)
ax.set_ylim(0.65,1.6)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)

ax.xaxis.set_tick_params(labelsize=14, width=2)
ax.yaxis.set_tick_params(labelsize=14, width=2)
ax.text(0.8, 1.35, r'$v_{acetone}$ = 0.52 ± 0.075, $R^{2}$ = 0.98', fontsize=16, color='#0371EC', **arfont)
ax.text(0.8, 1.31, r'$v_{DMSO}$ = 0.61 ± 0.010, $R^{2}$ = 0.99', fontsize=16, color='#418D2B', **arfont)
#plt.text(0.9, 1.23, r'$v_{acet-water}$ = 0.30 ± 0.16', fontsize=15, color='#562A8B', **arfont')
ax.text(0.8, 1.27, r'$v_{water}$ = 0.40 ± 0.032, $R^{2}$ = 0.99', fontsize=16, color='#6E563A', **arfont)

plt.show()

### Mean sq. e2e/Mean sq. Rg vs contour length 

In [ ]:
re2e_rgsq_plgapegwat = np.load('plga_peg_wat/PLGA-PEGwat_msRee_msRg.npy', allow_pickle=True)
re2e_rgsq_plgapegace = np.load('plga_peg_ace/PLGA-PEGace_msRee_msRg.npy', allow_pickle=True)
re2e_rgsq_plgapegdmso = np.load('plga_peg_dmso/PLGA-PEGdmso_msRee_msRg.npy', allow_pickle=True)

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.errorbar(re2e_rgsq_plgapegdmso[1]/10, re2e_rgsq_plgapegwat[0], yerr=re2e_rgsq_plgapegwat[2]/np.sqrt(5), color='slateblue', linestyle="solid",marker='o',
     capsize=5, capthick=1, ecolor='black')
ax.errorbar(re2e_rgsq_plgapegdmso[1]/10, re2e_rgsq_plgapegace[0], yerr=re2e_rgsq_plgapegace[2]/np.sqrt(5), color='darkslategrey', linestyle="dashed",marker='v',
     capsize=5, capthick=1, ecolor='black')
ax.errorbar(re2e_rgsq_plgapegdmso[1]/10, re2e_rgsq_plgapegdmso[0], yerr=re2e_rgsq_plgapegdmso[2]/np.sqrt(5), color='orangered', linestyle="dashdot",marker='^',
     capsize=5, capthick=1, ecolor='black')
#plt.title('Mean sq. Ree/Mean sq. Rg', fontsize=15, y=1.01)
ax.set_xlabel(r'Contour Length, $L_{c}$ [nm]', fontsize=16, **arfont)
ax.set_ylabel(r'$\left<R_{ee}^{2}\right>/\left<R_{g}^{2}\right>$', fontsize=16,  **arfont)
ax.legend(['PLGA-PEG water','PLGA-PEG acetone','PLGA-PEG DMSO'], frameon=False, fontsize=14)
ax.axhline(y=6, linewidth=1, color='b',ls='--')
ax.xaxis.set_tick_params(labelsize=14, width=2)
ax.yaxis.set_tick_params(labelsize=14, width=2)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
ax.set_xlim(1,15)
ax.set_ylim(2,10)

### Rg/Rh vs contour length 

In [ ]:
rhy_rg_plgapegwat = np.load('plga_peg_wat/PLGAPEGwater_RgRh_Lc.npy', allow_pickle=True)
rhy_rg_plgapegace = np.load('plga_peg_ace/PLGAPEGacetone_RgRh_Lc.npy', allow_pickle=True)
rhy_rg_plgapegdmso = np.load('plga_peg_dmso/PLGAPEGdmso_RgRh_Lc.npy', allow_pickle=True)

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.errorbar(rhy_rg_plgapegdmso[1]/10, rhy_rg_plgapegwat[0], yerr=rhy_rg_plgapegwat[2]/np.sqrt(5), color='slateblue', linestyle="solid",
             marker='o', capsize=5, capthick=1, ecolor='black')
ax.errorbar(rhy_rg_plgapegdmso[1]/10, rhy_rg_plgapegace[0], yerr=rhy_rg_plgapegace[2]/np.sqrt(5), color='darkslategrey', linestyle="dashed",
             marker='v', capsize=5, capthick=1, ecolor='black')
ax.errorbar(rhy_rg_plgapegdmso[1]/10, rhy_rg_plgapegdmso[0], yerr=rhy_rg_plgapegdmso[2]/np.sqrt(5), color='orangered', linestyle="dashdot",
             marker='^', capsize=5, capthick=1, ecolor='black')
#plt.title(r'Rg/Rh PLGA-PEG water', fontsize=15, y=1.01)
ax.set_xlabel(r'Contour Length, $L_{c}$ [nm]', fontsize=16,  **arfont)
ax.set_ylabel(r'$\left<R_{g}\right>/\left<R_{hyd}\right>$', fontsize=16,  **arfont)
ax.legend(['PLGA-PEG water','PLGA-PEG acetone','PLGA-PEG DMSO'], frameon=False, fontsize=14)
ax.set_xlim(1,15)
ax.set_ylim(0.3,0.8)
ax.xaxis.set_tick_params(labelsize=14, width=2)
ax.yaxis.set_tick_params(labelsize=14, width=2)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)

### Lp/Lc ratio

In [ ]:
kuhnlpL_plgapegwat = np.load('plga_peg_wat/PLGAPEGwater_lk_LpL.npy', allow_pickle=True)
kuhnlpL_plgapegace = np.load('plga_peg_ace/PLGAPEGacetone_lk_LpL.npy', allow_pickle=True)
kuhnlpL_plgapegdmso = np.load('plga_peg_dmso/PLGAPEGDMSO_lk_LpL.npy', allow_pickle=True)

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.errorbar(kuhnlpL_plgapegwat[1], kuhnlpL_plgapegwat[0], yerr=kuhnlpL_plgapegwat[2], color='slateblue', linestyle="solid",
             marker='o', capsize=5, capthick=1, ecolor='black')
ax.errorbar(kuhnlpL_plgapegace[1], kuhnlpL_plgapegace[0], yerr=kuhnlpL_plgapegace[2], color='darkslategrey', linestyle="dashed",
             marker='v', capsize=5, capthick=1, ecolor='black')
ax.errorbar(kuhnlpL_plgapegdmso[1], kuhnlpL_plgapegdmso[0], yerr=kuhnlpL_plgapegdmso[2], color='orangered', linestyle="dashdot",
             marker='^', capsize=5, capthick=1, ecolor='black')
ax.tick_params(axis='both',labelsize=15, width=1)

ax.set_xlabel(r'$N_{PLGA-PEG}$', fontsize=16, **arfont)
ax.set_ylabel(r'$L_{p}/L_{c}$', fontsize=16, **arfont)
ax.legend(['PLGA-PEG in water','PLGA-PEG in acetone','PLGA-PEG in DMSO'], frameon=False, fontsize=14)
ax.set_ylim(0,2)
ax.set_xlim(5,32)
ax.xaxis.set_tick_params(labelsize=14, width=2)
ax.yaxis.set_tick_params(labelsize=14, width=2)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)

### Mean sq. Ree vs contour length

In [ ]:
REElclk_plgapegwat = np.load('plga_peg_wat/PLGAPEGwater_REElclk.npy', allow_pickle=True)
REElclk_plgapegace = np.load('plga_peg_ace/PLGAPEGacetone_REElclk.npy', allow_pickle=True)
REElclk_plgapegdmso = np.load('plga_peg_dmso/PLGAPEGDMSO_REElclk.npy', allow_pickle=True)

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.errorbar(REElclk_plgapegwat[1]/10, REElclk_plgapegwat[0], yerr=REElclk_plgapegwat[2]/np.sqrt(5), color='slateblue', linestyle="None",
             marker='o', capsize=5, capthick=1, ecolor='black')
ax.errorbar(REElclk_plgapegace[1]/10, REElclk_plgapegace[0], yerr=REElclk_plgapegace[2]/np.sqrt(5), color='darkslategrey', linestyle="None",
             marker='v', capsize=5, capthick=1, ecolor='black')
ax.errorbar(REElclk_plgapegdmso[1]/10, REElclk_plgapegdmso[0], yerr=REElclk_plgapegdmso[2]/np.sqrt(5), color='orangered', linestyle="None",
             marker='^', capsize=5, capthick=1, ecolor='black')
ax.plot(REElclk_plgapegwat[1]/10, REElclk_plgapegwat[3],color='slateblue')
ax.plot(REElclk_plgapegace[1]/10, REElclk_plgapegace[3],color='darkslategrey')
ax.plot(REElclk_plgapegdmso[1]/10, REElclk_plgapegdmso[3],color='orangered')
#plt.title(r'Kuhn Length, PLGA-PEG water', fontsize=15)
ax.set_ylabel(r'$\left<R_{ee}^{2}\right>$', fontsize=16, **arfont)
ax.set_xlabel(r'Contour Length, $L_{c}$ [nm]', fontsize=16, **arfont)
ax.text(2.5, 1750, r'$l^{water}_{k}$ = 0.31 ± 0.36 nm, $R^{2}$ = 0.98', fontsize=15, color='slateblue',  **arfont)
ax.text(2.5, 1640, r'$l^{acetone}_{k}$ = 0.39 ± 0.12 nm, $R^{2}$ = 0.90', fontsize=15, color='darkslategrey',  **arfont)
ax.text(2.5, 1520, r'$l^{DMSO}_{k}$ = 1.26 ± 0.10 nm, $R^{2}$ = 0.98', fontsize=15, color='orangered',  **arfont)
ax.set_xlim(1,15)
ax.set_ylim(0,2000)
ax.xaxis.set_tick_params(labelsize=14, width=2)
ax.yaxis.set_tick_params(labelsize=14, width=2)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)

### Rg vs time in different sol per monomer length (Make sure to do it for Ree, Rhyd)